In [ ]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)